# 1. Working with Data
다음은 Working with data에 관련된 예제 코드이다. 데이터 작업을 위해서는 Dataset과 DataLoader가 필요하다. 

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

필요한 라이브러리들을 불러오는 과정이다. 

In [4]:
# 공개 데이터셋에서 학습 데이터를 내려받습니다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



데이터셋에는 여러가지 인자가 있는데, 그중에서도 샘플과 정답을 변경하기 위한 transform과 target_transform의 두 인자를 포함한다고 한다.

다음으로는 데이터셋을 데이터로더의 인자로 전달하는 과정이다.

In [5]:
batch_size = 64

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# 2. Creating Models


In [6]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


device 선언으로 CPU나 GPU 설정을 해준다.

다음으로 모델 class 선언을 하는데, 이는 nn.Module을 상속받아야 한다. 

init 함수 안에는 layer 정의를 해주고, foward에서 데이터가 어떻게 학습될지를 명시해놓는다. 

# 3. Optimizing the Model Parameters

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

Loss Function과 Optimizer를 지정해주는 코드

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

train 함수는 인자로 데이터로더, 모델, 로스펑션과 옵티마이저를 받는다. 

size는 데이터로더의 데이터셋의 길이로 지정해준뒤 train 함수를 실행시킨다. 

결과값과 로스값을 구한뒤, 백 프로퍼게이션을 실행시킨다. 

In [11]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303947  [    0/60000]
loss: 2.297262  [ 6400/60000]
loss: 2.272680  [12800/60000]
loss: 2.262936  [19200/60000]
loss: 2.258489  [25600/60000]
loss: 2.214709  [32000/60000]
loss: 2.229481  [38400/60000]
loss: 2.187070  [44800/60000]
loss: 2.189174  [51200/60000]
loss: 2.148230  [57600/60000]
Test Error: 
 Accuracy: 40.9%, Avg loss: 2.147664 

Epoch 2
-------------------------------
loss: 2.162578  [    0/60000]
loss: 2.150183  [ 6400/60000]
loss: 2.083254  [12800/60000]
loss: 2.099831  [19200/60000]
loss: 2.050614  [25600/60000]
loss: 1.982208  [32000/60000]
loss: 2.020074  [38400/60000]
loss: 1.925673  [44800/60000]
loss: 1.942575  [51200/60000]
loss: 1.855258  [57600/60000]
Test Error: 
 Accuracy: 58.9%, Avg loss: 1.857266 

Epoch 3
-------------------------------
loss: 1.897024  [    0/60000]
loss: 1.863073  [ 6400/60000]
loss: 1.731974  [12800/60000]
loss: 1.778220  [19200/60000]
loss: 1.670106  [25600/60000]
loss: 1.619543  [32000/600

# 4. Saving Models

In [12]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [13]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [14]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
